# Testeando el padData

## Paquetes necesarios

In [1]:
import transformData3 as td
import matplotlib.pyplot as plt
import numpy as np
import pycblosc2 as cb2
import time as t

## Definición del conjunto de datos

### Datos sin transformación

In [11]:
SHAPE = [126, 126, 126, 39]

SIZE = np.prod(SHAPE)

src = np.arange(SIZE, dtype=np.int32).reshape(SHAPE)

ITEMSIZE = src.dtype.itemsize

### Datos transformados

In [12]:
TSUBSHAPE = [32, 32, 32, 14]

srct, TSHAPE = td.tData(src, TSUBSHAPE, inverse=False)

In [13]:
padShape = []

for i in range(len(SHAPE)):
    d = 0
    if SHAPE[i] % TSUBSHAPE[i] != 0:
        d = TSUBSHAPE[i] - SHAPE[i] % TSUBSHAPE[i]
    padShape.append((0, d))

srcn = np.pad(src, padShape, 'constant')

In [14]:
np.testing.assert_array_equal(srct, srcn)